In [23]:
import uproot
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [97]:
path = "09Nov/"
name = "09Nov2021_testBeam_muons_1p2GSPS_nominalHV_min15_5k_angle30"
file = uproot.open(f"{path}{name}.root")

In [98]:
channel_map_waves = {
     "ch0" : [],
     "ch1" : [],
     "ch2" : [],
     "ch3" : [],
     "ch4" : [],
     "ch5" : [],
     "ch6" : [],
     "ch7" : [],
     "ch8" : [],
     "ch9" : [],
     "ch10" : [],
     "ch11" : [],
     "ch12" : [],
     "ch13" : [],
     "ch14" : [],
     "ch15" : []
}

channel_map_times = {
     "ch0" : [],
     "ch1" : [],
     "ch2" : [],
     "ch3" : [],
     "ch4" : [],
     "ch5" : [],
     "ch6" : [],
     "ch7" : [],
     "ch8" : [],
     "ch9" : [],
     "ch10" : [],
     "ch11" : [],
     "ch12" : [],
     "ch13" : [],
     "ch14" : [],
     "ch15" : []
}

In [99]:
import time, sys
from IPython.display import clear_output

def update_progress(progress, info=""):
    bar_length = 20
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))

    clear_output(wait = True)
    text = "{0} - Progress: [{1}] {2:.1f}%".format( info,"#" * block + "-" * (bar_length - block), progress * 100)
    print(text)

In [100]:
totevents = 500
for ev in np.arange(0,totevents):
    for ch in np.arange(0,16):
        update_progress(ev/totevents, f"Loading maps")
        waveform = file[f"Event{ev}"][f"Wafeform{ch}"].array()[0]
        time = file[f"Event{ev}"][f"Time{ch}"].array()[0]
        channel_map_waves[f"ch{ch}"].append(waveform)
        channel_map_times[f"ch{ch}"].append(time)

Loading maps - Progress: [####################] 99.8%


In [ ]:
len(channel_map_times['ch8'])

In [101]:
df_w = pd.DataFrame.from_dict(channel_map_waves)
df_t = pd.DataFrame.from_dict(channel_map_times)

df_w.to_pickle(f"{path}{name}.pkl")

In [65]:
del channel_map_waves
del channel_map_times

# Read Pkl file

In [2]:
df_w = pd.read_pickle("./08Nov2021/muons_165GeV_angle60_delay725ns_GSPS1p2_8Nov_0249.pkl")

In [3]:
totevents = df_w.shape[0]

In [102]:
import scipy.signal as spsig
from matplotlib import figure

def plotSigInEvent(eventNumber = 0):
    channel_map_color = {
     "ch0" : "b",
     "ch1" : "b",
     "ch2" : "b",
     "ch3" : "b",
     "ch4" : "black",
     "ch5" : "black",
     "ch6" : "black",
     "ch7" : "black",
     "ch8" : "black",
     "ch9" : "black",
     "ch10" : "maroon",
     "ch11" : "maroon",
     "ch12" : "maroon",
     "ch13" : "darkorange",
     "ch14" : "darkorange",
     "ch15" : "darkorange"
    }
    if (eventNumber > df_w.shape[0]):
        print("Event Number out of range")
        return -90
    fig = figure.Figure(figsize=(24,20))
    axs = fig.subplots(4,4)  #, sharex="all", sharey="all")
    for ch in np.arange(0,15):
        Row = int(ch / 4)
        Column = ch % 4
        ax = axs[Row][Column]
        
        w = np.array(df_w[f"ch{ch}"][eventNumber]) - 0.45
        t = np.array(df_t[f"ch{ch}"][eventNumber])# np.linspace(0,8.5e-7,1024)
        
        relmin = spsig.argrelmin(w,order=50)
        #print(w,relmin)
        _ = ax.plot(t,w, channel_map_color[f"ch{ch}"])
        ax.set_ylim(-0.22,0.1)
        ax.tick_params(direction='in', length=6, width=1.1, colors='k',
                      grid_color='k', which='major')
        ax.tick_params(direction='in', length=3, width=1, colors='k',
                      grid_color='k', which='minor')
        ax.grid(ls='--',color='grey')

    return fig


In [103]:
import matplotlib.backends.backend_pdf
outpath = "PLOTS"
pdf = matplotlib.backends.backend_pdf.PdfPages(os.path.join(outpath,f"{name}.pdf"))

for ev in np.arange(0,totevents):
    update_progress(ev/totevents, f"Plotting")
    txt = f'Event={ev+1}'
    fig = plotSigInEvent(ev)
    fig.suptitle(txt)
    plt.text(0.05,0.95,txt, transform=fig.transFigure, size=24)
    #fig.savefig(os.path.join(outpath,"event_{0}.png".format(ev+1)))
    pdf.savefig( fig )
    plt.close()
pdf.close()

Plotting - Progress: [####################] 99.8%


In [ ]:
del fig

In [ ]:
from scipy.interpolate import CubicSpline
from matplotlib.gridspec import GridSpec


f_s = 1.2e9
dt = 1/f_s

size = 1024
d = np.array(df_w["ch13"][40])
    
## DFT using numpy routine

x  = np.linspace(0.0,size*dt, size)
yf = np.fft.fft(d)
xf = np.fft.fftfreq(size, dt)
xf = np.fft.fftshift(xf)
print(xf)
yplot = np.fft.fftshift(yf)
## Power Spectrum
spectrum = np.abs(yplot)**2
   
## Power Spectrum Cubic Spline Interpolation
cs = CubicSpline(xf, spectrum)
new_xf = np.linspace(xf.min(), xf.max(), 5760)
spectrum_cs = cs(new_xf)
plot=True
save=False
             
if plot:
    fig = plt.figure(figsize=(30,8))
    gs = GridSpec(1, 3, figure=fig)
    
    ax1 = fig.add_subplot(gs[0, :-1])
    color = 'xkcd:crimson'
    ax1.set_xlabel('time (s)')
    ax1.set_ylabel('V')
    ax1.tick_params(axis='y')#, labelcolor=color)
    ax1.plot(np.linspace(0,d.size,size)*dt,d, color=color, ls='--')
 
    ax2 = fig.add_subplot(gs[0, -1])
    ax2.plot(xf, 1.0/size * spectrum)
    #ax2.plot(new_xf, 1.0/size * spectrum_cs,':')
    ax2.set_xlabel("f [Hz]")
    ax2.set_title("Power Spectrum")
    ax2.set_xlim(0,0.6e9)
    ax2.set_yscale('log')
    if save:
        plt.savefig(f"fft_window_{tag}.png", bbox_inches='tight')
        plt.close()


In [ ]:
sp.fft(df_w[f"ch7"][42])